## 第6章 実務に役立つアルゴリズムを知る  
#### keyword:

### 6.1 最短経路問題とは？

時間・費用・距離などをそれぞれ数値化してコストとして考える。

格子状の経路で、最短経路がいくつあるか求める手法
1. 右にm回, 上にn回移動する場合m+nCm
2. 交点を通るパターン数を左下から足していく → プログラミングでも便利  
- 動的計画法（DP）

In [ ]:
# near_route1.py

M, N = 6, 5

route = [[0 for i in range(N + 1)] for j in range(M + 1)]

for i in range(M + 1):
    route[i][0] = 1
    
for i in range(1, N + 1):
    route[0][i] = 1
    for j in range(1, M + 1):
        route[j][i] = route[j - 1][i] + route[j][i - 1]
        
print(route[M][N])

- メモ化(計算済の情報を保持)

In [ ]:
# near_route2.py

import functools

M, N = 6, 5

@functools.lru_cache(maxsize = None)
def search(m, n):
    if (m == 0) or (n == 0):
        return 1
    
    return search(m - 1, n) + search(m, n - 1)

print(search(M, N))

### *20210205*

### 6.2 ベルマンフォード法

- 辺の重みに注目して解く  
- スタート地点は0, それ以外のコストの初期値は無限大`float('inf')`  

In [ ]:
# bellman_ford.py

def bellman_ford(edges, num_v):
    dist = [float('inf') for i in range(num_v)]
    dist[0] = 0
    
    changed = True #コストが更新されたか
    while changed:
        changed = False
        for edge in edges:
            if dist[edge[1]] > dist[edge[0]] + edge[2]:
                dist[edge[1]] = dist[edge[0]] + edge[2]
                changed = True
                
    return dist

# １つの辺は起点と終点の番号、コストの３つの要素をもつ
edges = [
    [0, 1, 4], [0, 2, 3], [1, 2, 1], [1, 3, 1],
    [1, 4, 5], [2, 5, 2], [4, 6, 2], [5, 4, 1],
    [5, 6, 4]
]

print(bellman_ford(edges, 7))

### 6.3 ダイクストラ法

コストが最小になる頂点を選択することを繰り返す

In [ ]:
# dijkstra.py

def dijkstra(edges, num_v):
    dist = [float('inf')] * num_v
    dist[0] = 0
    q = [i for i in range(num_v)]
    
    while len(q) > 0:
        r = q[0]
        for i in q:
            if dist[i] < dist[r]:
                r = i
                
        u = q.pop(q.index(r))
        for i in edges[u]:
            if dist[i[0]] > dist[u] + i[1]:
                dist[i[0]] = dist[u] + i[1]
                
    return dist

edges = [
    [[1, 4], [2, 3]],
    [[2, 1], [3, 1], [4, 5]],
    [[5, 2]],
    [[4, 3]],
    [[6, 2]],
    [[4, 1], [6, 4]],
    []
]

print(dijkstra(edges, 7))